In [17]:
import os
import json
import numpy as np
import pandas as pd
from copy import deepcopy

### GT data info and Attention Check

In [18]:
gt_df = pd.read_csv('../build/data/static.pilot_set-1.csv')
label_map = {"not sarcastic": 0, "sarcastic": 1}
gt_df['label'] = gt_df['original_label'].map(label_map)

In [19]:
gt_df

,original_data,original_label,label
0,I think our cat has separation anxiety.,not sarcastic,0
1,Clapped out the post twice now.,not sarcastic,0
2,Lucky I'm not going to that then…,not sarcastic,0
3,nevermind i can't put them in,not sarcastic,0
4,Not to be a spelling nerd but please check you...,not sarcastic,0
5,Now is an amazing time to join the group. Musi...,not sarcastic,0
6,If you don't have a PHD in immunology then you...,not sarcastic,0
7,sometimes i wish i could just do the most obvi...,not sarcastic,0
8,New years resolution this year is to pick up s...,not sarcastic,0
9,It's like the whole world is becoming woke to ...,not sarcastic,0


In [20]:
ATTENTION_CHECK_SIG = 'Please determine if the following statement is true or false.'
ATTENTION_CHECK_ANSWERS = {
    '1 + 1 = 2': True,
    'Texas is the capital of the United States.': False,
    'The sun rises from the north and sets at the west.': False,
}

## Load human data

In [21]:
human_result_path = 'raw_data.json'
human_data = json.load(open(human_result_path, 'r'))

In [26]:
# get individual subject data and convert to pandas dataframe
subject_dfs = {}
subject_attention_check = {}
subject_regular_questions = {}

for subject_id, data in human_data.items():
    if subject_id != 'surveys':
        print(f"Subject: {subject_id}")
        df = pd.DataFrame.from_dict(data)
        attention_check_results = []
        regular_question_results = []
        
        for i, row in df.iterrows():            
            statement = row['statement']
            response = True if row['response'] == 'True' else False
            if row['question'] == ATTENTION_CHECK_SIG:            
                correct_answer = ATTENTION_CHECK_ANSWERS[statement]
                attention_check_results.append(response == correct_answer)                
            else:
                correct_answer = gt_df[gt_df['original_data'] == statement]['label'].values[0]                
                regular_question_results.append(response == correct_answer)

        subject_dfs[subject_id] = deepcopy(df)
        subject_attention_check[subject_id] = deepcopy(attention_check_results)
        subject_regular_questions[subject_id] = deepcopy(regular_question_results)        
        print(f"Attention check results: {np.mean(attention_check_results)}")
        print(f"Regular question results: {np.mean(regular_question_results)}")
    

Subject: 5f529c8d81da7d58cd46cc29
Attention check results: 0.6666666666666666
Regular question results: 0.65
Subject: honeydew
Attention check results: 1.0
Regular question results: 0.75
